<a href="https://colab.research.google.com/github/BayodeOlorundare/kaggle_api_python/blob/main/2022_fortune_1000_kaggle_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the Kaggle Python library

In [29]:
! pip install kaggle

Mount the Google Drive so you can store your Kaggle API credentials for future use

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Make a directory for kaggle at the temporary instance location on Colab drive.

Download your kaggle API key (.json file). You can do this by going to your kaggle account page and clicking 'Create new API token' under the API section.

In [31]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Upload the json file to Google Drive and then copy to the temporary location.

In [32]:
!cp /content/drive/MyDrive/Work/Training/Projects/kaggle.json ~/.kaggle/kaggle.json


Change the file permissions to read/write to the owner only

In [33]:
! chmod 600 ~/.kaggle/kaggle.json

Download dataset from Kaggle

In [34]:
! kaggle datasets download -d winston56/fortune-500-data-2021

fortune-500-data-2021.zip: Skipping, found more recently modified local copy (use --force to force download)


Unzip, in case the downloaded file is zipped. Refresh the files on the left hand side to update the view.

In [35]:
! unzip fortune-500-data-2021.zip

Archive:  fortune-500-data-2021.zip
replace Fortune_1000.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Fortune_1000.csv        


Using Panda, we will pivot the data so we can create multiple levels for a sunburst when creating visualiztion in Tableau.

Then upload the changed csv file into Google Drive folder.

In [36]:
import pandas as pd

# Step 1: Read the CSV file and create the DataFrame
df = pd.read_csv('/content/Fortune_1000.csv')

# Step 2: Group by 'sector' and calculate the total profit for each sector
sector_totals = df.groupby('sector')['profit'].sum().reset_index()

# Step 3: Append a new row for each sector with the total profit for all companies
sector_totals['company'] = ''
df = pd.concat([df, sector_totals], ignore_index=True)

# Step 4: Sort the DataFrame by 'sector' to have the 'Total' row grouped with each sector
df = df.sort_values(by='sector')

# Step 5: Reset the index to maintain a clean DataFrame
df = df.reset_index(drop=True)

# Step 6: Add the 'level' column
df['level'] = df['company'].apply(lambda x: 1 if x == '' else 2)

# Step 7: Sort the DataFrame by sector and profit in descending order
df_sorted = df.sort_values(by=['sector', 'profit'], ascending=[True, False])

# Step 8: Group the sorted DataFrame by sector
grouped = df_sorted.groupby('sector')

# Step 9: Select the top 2 companies in each sector based on highest profit
df_top_2 = grouped.apply(lambda x: x.nlargest(2, 'profit')).reset_index(drop=True)

# Step 10: Duplicate the original dataset
df_duplicated = pd.concat([df_top_2] * 2, ignore_index=True)

# Step 11: Add the 'ToPad' column and set the values
df_duplicated['ToPad'] = 1
df_duplicated.loc[df_duplicated.index >= len(df_top_2), 'ToPad'] = 203

# Step 12: Save the changes back to the original CSV file
# Provide the correct file path to your CSV file in Google Drive to save to
csv_file_path = '/content/drive/MyDrive/Work/Training/Projects/Infographics/Infographic Data/Fortune_1000.csv'
df_duplicated.to_csv(csv_file_path, index=False)  # Set index=False to exclude index column from saving

print("Changes saved to the CSV file. And uploaded to Google Drive!")

Changes saved to the CSV file. And uploaded to Google Drive!
